# Code for reproducing plots in "Did we personalize? Assessing personalization by an online reinforcement learning algorithm using resampling"

In [7]:
## Setup
import subprocess
import numpy as np
import sys
import os
from datetime import date, datetime
from itertools import product
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import pickle as pkl
import seaborn as sns

import matplotlib as mpl
import pylab
from matplotlib import rc

lsize=80
axSize=67

def setPlotSettings(fix_plot_settings=True):
    if fix_plot_settings:
        plt.rc('font', family='serif')
        plt.rc('text', usetex=True)
        plt.rcParams['text.usetex'] = True
        label_size = lsize
        mpl.rcParams['axes.labelsize'] = label_size
        mpl.rcParams['axes.titlesize'] = label_size
        mpl.rcParams['xtick.labelsize'] = axSize 
        mpl.rcParams['ytick.labelsize'] = axSize 
        mpl.rcParams['figure.titlesize'] = label_size
        mpl.rcParams['lines.markersize'] = label_size
        mpl.rcParams['grid.linewidth'] = 2.5
        mpl.rcParams['legend.fontsize'] = label_size
    
        pylab.rcParams['xtick.major.pad']=5
        pylab.rcParams['ytick.major.pad']=5

        lss = ['--',  ':', '-.', '-', '--', '-.', ':', '-', '--', '-.', ':', '-']
        mss = ['>', 'o',  's', 'D', '>', 's', 'o', 'D', '>', 's', 'o', 'D']
        ms_size = [25, 20, 20, 20, 20, 20, 20, 20, 20, 20]
        colors = ['#e41a1c', '#0000cd', '#4daf4a',  'black' , 'magenta']
    
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.7g" % x))

NUSERS = 91
F_KEYS=["intercept", "dosage", "engagement", "other_location", "variation"]

## Int Score Computation, at day level sliding window
def computeMetricSlidingDay(result, indexFS, x=2, delta1=.5, delta2=.5, IGNORE_ETA=False):
    ndata=rindex(result['availability'][:T], 1)+1 # +1 for actual number of timepoints. 
    if ndata==0: #if no one is found
        print("no availability :(")
        return {'isEqualAvail':False, 'isEqualEngAvail':False, 'r1':None, 'r2':None, 'stdEffectRatio': None}

    last_day=math.floor((ndata-1)/NTIMES) #ndata is the ts index of the last available time
    
    # Generate list of standardized adv forecasts for this user.
    varValues=[] #binary 0/1 for \var (z)
    stdEffects=[] #the standardized effects for the user (\hat{\Delta}(\cdot))
    etas=[] # eta values

    effLastDay=min(last_day+1+1,90)
    for day in range(effLastDay): #want it to iterate to last day so +1. One more in case we stop before day 90 and can still forecast next day.
        for time in range(NTIMES):
            ts = (day) * 5 + time
            if result['availability'][ts]==1.:
                ## get mean/std and eta
                # the one at ts is really the posteriors from the last day, or (ts//5)-1 since we update at end of the day. 
                # Consequently, the values at day 0, or times 0-4, correspond to the prior params
                beta=result['post_beta_mu'][day*5][-len(F_KEYS):] 
                mean =result['fs'][ts] @ beta

                sigma=result['post_beta_sigma'][day*5][-len(F_KEYS):, -len(F_KEYS):]
                std=math.sqrt((result['fs'][ts] @ sigma.T) @ result['fs'][ts])
                
                eta=0 if IGNORE_ETA else result['etas'][ts]
                
                ## compute stdEffect
                etas.append(eta)
                stdEffect=(mean-eta)/std

                stdEffects.append(stdEffect)
                varValues.append(result['fs'][ts, indexFS])
            else: #if not available put none
                stdEffects.append("NONE")
                varValues.append("NONE")

    varValues=np.array(varValues)

    # for computing int scores, track a few variables
    ## int score 2, \var
    nSlidingWindows=NDAYS
    nInterestingDeterminedWindows=0
    nDeterminedWindows=0
    
    ## int score 1
    nSlidingWindows_intscore1=NDAYS
    nInterestingDeterminedWindows_intscore1=0
    nDeterminedWindows_intscore1=0

    avgVarEffAll=[]
    avgNonVarEffAll=[]
    determinedTimes=[]
    
    # loop through each day, and (1) form sliding windows, (2) check for G_{d,1}, (3) compute int_score
    for day in range(effLastDay):
        avail_idx_pre2=np.array([]) # var used for determining if an update occurred 2 days before?
        avail_idx_pre1=np.array([]) # --""-- 1 day before
        avail_idx_cur=np.array([]) # --""-- current day
        if day == 0: #length of sliding window is 2*NTIMES
            startTime=0
            endTime=NTIMES*2

            # check day 0 has any updates
            avail_idx_cur = np.logical_and(~np.isnan(result['reward'][0:NTIMES]), result['availability'][0:NTIMES] == 1)

        elif day >= 1 and day < last_day: #length of sliding window is 3*NTIMES
            startTime=day*NTIMES-NTIMES
            endTime=day*NTIMES+NTIMES*2

            if day>=2:
                avail_idx_pre2 = np.logical_and(~np.isnan(result['reward'][(day*NTIMES-2*NTIMES):(day*NTIMES-NTIMES)]), result['availability'][(day*NTIMES-2*NTIMES):(day*NTIMES-NTIMES)] == 1)
            avail_idx_pre1 = np.logical_and(~np.isnan(result['reward'][(day*NTIMES-NTIMES):(day*NTIMES)]), result['availability'][(day*NTIMES-NTIMES):(day*NTIMES)] == 1)
            avail_idx_cur = np.logical_and(~np.isnan(result['reward'][(day*NTIMES):(day*NTIMES+NTIMES)]), result['availability'][(day*NTIMES):(day*NTIMES+NTIMES)] == 1)

        else: #if last_day, length of sliding window is 2*NTIMES
            startTime=day*NTIMES-NTIMES
            endTime=day*NTIMES+NTIMES
            
            # check day lastday-1 has any updates
            avail_idx_pre2 = np.logical_and(~np.isnan(result['reward'][(day*NTIMES-2*NTIMES):(day*NTIMES-NTIMES)]), result['availability'][(day*NTIMES-2*NTIMES):(day*NTIMES-NTIMES)] == 1)
            avail_idx_pre1 = np.logical_and(~np.isnan(result['reward'][(day*NTIMES-NTIMES):(day*NTIMES)]), result['availability'][(day*NTIMES-NTIMES):(day*NTIMES)] == 1)
            if day < 89:
                endTime=day*NTIMES+2*NTIMES
                avail_idx_cur = np.logical_and(~np.isnan(result['reward'][(day*NTIMES):(day*NTIMES+NTIMES)]), result['availability'][(day*NTIMES):(day*NTIMES+NTIMES)] == 1)

        ## Subset above varValues and forecasts to sliding window durations
        varWindow=varValues[startTime:endTime]
        forecastsWindow=stdEffects[startTime:endTime]

        # check that an update happened before any of the day windows in question. 
        enoughUpdates = (sum(avail_idx_pre2)>0) or (sum(avail_idx_pre1)>0) or (sum(avail_idx_cur)>0) #a function of non observed reward. 
        
        # G_{d,1} condition
        varIndices=np.where(varWindow==1)[0]
        nonVarIndices=np.where(varWindow==0)[0]
        nBlue=len(varIndices)
        nRed=len(nonVarIndices)
        isDetermined = (nBlue >=x) and (nRed >= x) and enoughUpdates

        #if G_{d,1} for intscore 2
        if isDetermined: 
            nDeterminedWindows=nDeterminedWindows+1
            determinedTimes.append(day)

            # calculate day-level avg forecasts
            avgVarEffect=np.mean(forecastsWindow[varIndices])
            avgNonVarEffect=np.mean(forecastsWindow[nonVarIndices])

            avgVarEffAll.append(avgVarEffect)
            avgNonVarEffAll.append(avgNonVarEffect)
            
            # compare to determine if an 'interesting' window
            if avgVarEffect > avgNonVarEffect:
                nInterestingDeterminedWindows=nInterestingDeterminedWindows+1

        # if G_{d,1} for intscore 1
        if sum(avail_idx_pre1) > 0 or day==0: # if an update occurred the day prior
            effects_intscore1=stdEffects[(day*NTIMES):(day*NTIMES+NTIMES)][effects_intscore1!="None"]
            #effects_intscore1=effects_intscore1[effects_intscore1!="None"]
            if len(effects_intscore1)>=2: #if we have enough data in the day window
                nDeterminedWindows_intscore1=nDeterminedWindows_intscore1+1
                if np.mean(effects_intscore1)>0: # if the window is interesting enough
                    nInterestingDeterminedWindows_intscore1=nInterestingDeterminedWindows_intscore1+1

    nUndeterminedSlidingWindows=nSlidingWindows-nDeterminedWindows
    nUndeterminedSlidingWindows_intscore1=nSlidingWindows_intscore1-nDeterminedWindows_intscore1

    # output int scores (one and two sided) and G_{d,1} fractions
    statistic={}
    # int score 2
    if nSlidingWindows >0 and nDeterminedWindows >0:
        statistic["r1"]=nUndeterminedSlidingWindows/nSlidingWindows
        statistic["rawR2"]=nInterestingDeterminedWindows/nDeterminedWindows
        statistic["r2"]=abs(nInterestingDeterminedWindows/nDeterminedWindows - 0.5)
        statistic["isInteresting_2"]=(statistic["r1"]<=delta1) and (statistic["r2"]>=delta2)
    else: 
        statistic["r1"]=None
        statistic["r2"]=None
        statistic["rawR2"]=None
        statistic["isInteresting_2"]=None
        
    # int score 1
    if nSlidingWindows_intscore1>0 and nDeterminedWindows_intscore1 > 0:
        statistic["r3"]=nUndeterminedSlidingWindows_intscore1/nSlidingWindows_intscore1#modified to just be for if there are enough updates.
        statistic["rawR4"]=nInterestingDeterminedWindows_intscore1/nDeterminedWindows_intscore1 #
        statistic["r4"]=abs(nInterestingDeterminedWindows_intscore1/nDeterminedWindows_intscore1 - 0.5) #
        statistic["isInteresting_1"]=(statistic["r3"]<=delta1) and (statistic["r4"]>=delta2)
    else:
        statistic["r3"]=None
        statistic["r4"]=None
        statistic["rawR4"]=None
        statistic["isInteresting_1"]=None
    
    # to reproduce twin curves of avg of engaged and not engaged effects at determiend times
    statistic["determinedTimes"]=determinedTimes
    statistic["avgNonValAll"]=avgNonEngageAll
    statistic["avgValAll"]=avgEngageAll
    
    # to reproduce plot of standardized posterior at engaged and not engaged states
    statistic['varValues']=varValues
    statistic['standardizedEffects']=stdEffects
    statistic['etas']=etas
    return statistic


def plot_C1_histogram(r1s, baseline):
    image="./plots_test"+'/histogram_C1_'+baseline+'.pdf'
    setPlotSettings(True)
    fig, ax = plt.subplots(figsize=(15, 15))
    barcol='gray'
    bins="auto"
    df=pd.DataFrame(r1s, columns=['gd1'])
    p = sns.histplot(data=df, x='gd1', bins=bins, stat='count', ax=ax, color=barcol, cbar_kws={"linewidth":0}, line_kws={"linewidth":0}, linewidth=0)

    # cosmetics
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    plt.xlabel("")
    plt.ylabel("\# Users")
    plt.grid(axis='y', alpha=.5, zorder=0) 
    ax=plt.gca()
    
    from matplotlib.ticker import MaxNLocator
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    yticks = ax.yaxis.get_major_ticks()
    yticks[0].label1.set_visible(False)
    
    plt.xlim([0,1])
    ax.set_xticklabels([0,.25,.5,.75,1])
    plt.tight_layout()
    plt.savefig(image, format="pdf", bbox_inches="tight")
    
    print(image)
    plt.clf()
    return


def plot_C2_histogram(r2s, baseline):
    image="./plots_test"+'/histogram_C2_'+baseline+'.pdf'
    setPlotSettings(True)
    plt.rcParams['text.usetex'] = True
    fig, ax = plt.subplots(figsize=(15, 15))
    barcol='gray'
    df=pd.DataFrame(rawR2s, columns=['rawR2s'])
    bins=10
    p = sns.histplot(data=df, x='rawR2s', bins=bins, stat='count', ax=ax, color=barcol, cbar_kws={"linewidth":0}, line_kws={"linewidth":0}, linewidth=0)
    
    # cosmetics
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    plt.xlabel("")
    plt.ylabel("\# Users")
    plt.grid(axis='y', alpha=.5, zorder=0) 
    ax=plt.gca()
    
    from matplotlib.ticker import MaxNLocator
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    yticks = ax.yaxis.get_major_ticks()
    yticks[0].label1.set_visible(False)
    
    plt.xlim([0,1])
    ax.set_xticklabels([0,.25,.5,.75,1])
    plt.axvline(.1, color='k', ls='--', zorder=4, lw=6)
    plt.axvline(.9, color='k', ls='--', zorder=4, lw=6)
    
    plt.tight_layout()
    plt.savefig(image, format="pdf", bbox_inches="tight")
    
    print(image)
    plt.clf()
    return

def plot_heatmap_histogram(ogR1, ogR2, baseline, B=500):
    # range of \gamma and \delta to go over
    gammas=[.75, .70,.65]
    deltas=[.35,.40,.45]
    
    heatMap=[]
    heatMapR1=[] # g_{d,1}
    output_dir = os.path.join("./output", "Baseline-"+ str(baseline)+"_UserSpecific-"+str("False"))

    for gamma in gammas:
        heatMapRow=[]
        heatMapRowR1=[]
        for delta in deltas:
            bootstrapNInteresting=[] 
            observed=sum(np.logical_and(ogR1 <= gamma, ogR2 >= delta))
            
            for bootstrap in range(B): #
                output_dirB = os.path.join(output_dir, "Bootstrap-" + str(bootstrap))
                results=os.path.join(output_dirB, "results.csv")
                df=pd.read_csv(results)

                bootstrapR1s=np.array(df[r1Key])
                if 'None' in bootstrapR1s:
                    bootstrapR1s=bootstrapR1s[bootstrapR1s.astype(str) != 'None']
                bootstrapR1s=bootstrapR1s.astype(float)

                bootstrapR2s=np.array(df[r2Key])
                if 'None' in bootstrapR2s:
                    bootstrapR2s=bootstrapR2s[bootstrapR2s.astype(str) != 'None']
                bootstrapR2s=bootstrapR2s.astype(float)

                bootstrapValue=sum(np.logical_and(bootstrapR1s <= delta1, bootstrapR2s >= delta2))
                bootstrapNInteresting.append(bootstrapValue)

            perc=stats.percentileofscore(bootstrapNInteresting, observed, 'weak')/100
            heatMapRow.append(1-perc)
            heatMapRowR1.append(sum(ogR1<=gamma))

            #plot histogram too!
            if delta1==.75 and delta2==.4: # can generate for other values too
                image="./plots_test"+'/histogram_Interesting_'+baseline+"_delta1="+str(gamma)+"_delta2="+str(delta)+"_B="+str(B)+'.pdf'
                plt.clf()
                setPlotSettings(True)
                fig, ax = plt.subplots(figsize=(15, 15))
                barcol='gray'
                bins="auto"
                df=pd.DataFrame(bootstrapNInteresting, columns=['nInt'])
                p = sns.histplot(data=df, x='nInt', bins=bins, stat='probability', ax=ax, color=barcol, cbar_kws={"linewidth":0}, line_kws={"linewidth":0}, linewidth=0)
                
                # cosmetics
                for spine in ['top', 'right']:
                    ax.spines[spine].set_visible(False)
                plt.xlabel("")
                plt.ylabel("Proportion")
                plt.grid(axis='y', alpha=.5, zorder=0)
                plt.axvline(observed, color='b', ls='-.', zorder=4, lw=6)
                plt.xlim(left=0)
                if baseline=="Zero" and delta1==.75 and delta2==.4:
                    labs=['0', '0.02','0.04','0.06','0.08','0.10','0.12','0.14']
                    vals=[float(lab) for lab in labs]
                    labs[0]=''
                    plt.yticks(vals,labs)
                elif baseline=="other_location" and delta1==.75 and delta2==.4:
                    labs=['0', '0.05','0.10','0.15','0.20','0.25','0.30','0.35']
                    vals=[float(lab) for lab in labs]
                    labs[0]=''
                    plt.yticks(vals,labs)
                elif baseline=="variation" and delta1==.75 and delta2==.4:
                    labs=['0', '0.025','0.050','0.075','0.100','0.125','0.150','0.175']
                    vals=[float(lab) for lab in labs]
                    labs[0]=''
                    plt.yticks(vals,labs)
                else:
                    yticks = ax.yaxis.get_major_ticks() 
                    yticks[0].label1.set_visible(False)
                plt.tight_layout()
                plt.savefig(image, format="pdf", bbox_inches="tight")
                print(image)
        
        heatMap.append(heatMapRow)
        heatMapR1.append(heatMapRowR1)

    # Now plot the heatmap!
    sns.set(rc={'text.usetex': True})
    plt.clf()
    setPlotSettings(True)
    fig, ax=plt.subplots(figsize=(15,15))
    
    heatMap=pd.DataFrame(heatMap)
    heatMapR1=pd.DataFrame(heatMapR1)
    
    gammaLabels=['0.75', '0.70','0.65']
    deltaLabels=['0.35','0.40','0.45']
    s=sns.heatmap(heatMap,xticklabels=deltaLabels, ax=ax, yticklabels=gammaLabels, cmap="Blues_r", fmt='.2f', annot=True, vmin=0, vmax=1, annot_kws={'fontsize': lsize})
    plt.yticks(rotation=0)
    s.set(xlabel="$\delta$", ylabel="$\gamma$")
    f=s.get_figure()
    
    toWrite=baseline
    if baseline=="other_location":
        toWrite="other location"
    image="./plots_test"+'/heatMap_Interesting_'+toWrite+"_B="+str(B)+'.pdf'
    print(image)
    plt.tight_layout()
    f.savefig(image, format="pdf", bbox_inches="tight")
    plt.clf()

# get original int scores and G_{d,1} ratios.
def getOriginalResults(indexFS, delta=.4, gamma=.75):
    baseline="Zero"
    if indexFS!=-1:
        baseline=F_KEYS[indexFS]

    original_result="./init/original_result_91.pkl"
    with open(original_result, 'rb') as handle:
        original_result=pkl.load(handle)

    r1s=[]
    r2s=[]
    rawR2s=[]
    interesting=[]
    
    for result in original_result:
        result=computeMetricSlidingDay(result, experiment, delta1=gamma, delta2=delta)

        # get proper results
        r1=result['r1']
        r2=result['r2']
        rawR2=result['rawR2']
        interestingResult=result['isInteresting_2']
        if baseline=="Zero":
            r1=result['r3']
            r2=result['r4']
            rawR2=result['rawR4']
            interestingResult=result['isInteresting_1']

        # store relevant values
        if r1 != None:
            r1s.append(r1)
            if r1 <= delta1: # if they are worth consideration (have enough data)
                r2s.append(r2)
                rawR2s.append(rawR2)
        interesting.append(interestingResult)
        
    r1s=np.array(r1s)
    r2s=np.array(r2s)
    rawR2s=np.array(rawR2s)
    
    # now r2s by users
    r2Users={}
    r2RawUsers={}
    for i in users:
        r2Users[str(i)]=[]
        r2RawUsers[str(i)]=[]
        output_dir = os.path.join("./output", "Baseline-"+ str(baseline)+"_UserSpecific-"+str(True)+"_User-" + str(i))
        results=os.path.join(output_dir, "results.csv")
        results=pd.read_csv(results)
        r2RawUsers[str(i)]=results[r2RawKey]
        r2Users[str(i)]=results[r2Key]

    return r1s,r2s,rawR2s,interesting, r2Users, r2RawUsers, original_result

def plot_r2_User(r2Users, r2RawUsers, rawR2, interesting, baseline, users):
    percs=[]
    for i in range(len(users)):
        user=users[i]
        if interesting[user]==True: #only added if its interesting
            perc=stats.percentileofscore(r2Users[str(i)], float(holderR2s[i]), 'weak')/100
            percs.append(1-perc)

    #plot histogram of Frac
    image="./plots_test"+'/histogram_percs_'+baseline+'_x=2'+'.pdf'
    plt.clf()
    setPlotSettings(True)
    fig, ax=plt.subplots(figsize=(15,15))
    plt.grid(axis='y', alpha=.5, zorder=0)
    barcol="gray"
    df=pd.DataFrame(percs, columns=['percs'])
    p = sns.histplot(data=df, x='percs', stat='count', ax=ax, color=barcol, cbar_kws={"linewidth":0}, line_kws={"linewidth":0}, linewidth=0)
    
    # cosmetics
    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    ax=plt.gca()
    from matplotlib.ticker import MaxNLocator
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    yticks = ax.yaxis.get_major_ticks()
    yticks[0].label1.set_visible(False)
    
    plt.xlabel("")
    plt.ylabel("\# Users")

    plt.tight_layout()
    plt.savefig(image, format="pdf",bbox_inches="tight")
    print(image)
    plt.clf()
    
    # Now plot histogram of int scores!
    for i in users:
        if ((i == 4 and baseline=="variation") or (i==77 and baseline=="Zero")):
            image="./plots_test"+'/histogram_c2s_user_'+str(i)+'_'+baseline+'_x=2'+'.pdf'
            plt.clf()
            r2s=r2RawUsers[str(i)]
            
            setPlotSettings(True)
            fig, ax=plt.subplots(figsize=(24,16))#, dpi=80)
            plt.grid(axis='y', alpha=.5, zorder=0)
            barcol="gray"
            df=pd.DataFrame(np.array(r2s), columns=['r2s'])
            p = sns.histplot(data=df, x='r2s', stat='probability', ax=ax, color=barcol, cbar_kws={"linewidth":0}, line_kws={"linewidth":0}, linewidth=0)
            
            # cosmetics
            for spine in ['top', 'right']:
                ax.spines[spine].set_visible(False)
            ax=plt.gca()
            plt.xlabel("")
            plt.ylabel("Proportion")
            plt.axvline(allRawR2[i], ls='-.', color='b', zorder=4, lw=6)
            plt.xlim(left=-.05)
            plt.xticks([0, .25,.5,.75,1])
            
            yticks = ax.yaxis.get_major_ticks()
            yticks[0].label1.set_visible(False)
            plt.tight_layout()
            plt.savefig(image, format="pdf", bbox_inches="tight")
            print(image)
            plt.clf()

            
def plotUserDayDateAndResims(result, resultRun, user, outputPath, stateName, bs_results, bs_resultsRuns, bs):
    pathPeng="./init/all91_uid.pkl"
    resultPeng=pkl.load(open(pathPeng, "rb"))
    uids=resultPeng[:,0,15]
    otherDF=pd.DataFrame(uids, range(resultPeng.shape[0]), columns=["StudyID"])
    otherDF['indices']=range(len(uids))

    pathBaselineInfo="./init/baseline_info.csv"
    baseline_info=pd.read_csv(pathBaselineInfo)
    baseline_info=baseline_info[['start.date', 'StudyID']] # match by UIDS!

    start_dates=pd.merge(otherDF, baseline_info, on="StudyID")

    # determined times, standardized effects
    rc('mathtext', default='regular')
    fig = plt.figure(figsize=(24, 16))
    ax = fig.add_subplot()#111)#, aspect='equal')
    spacing = .4

    ndata=rindex(resultRun['availability'][:T], 1)+1 # +1 for actual number of timepoints. #np.where(data[:T,1]==0)[0] #first time the user is out
    last_day=math.floor((ndata-1)/NTIMES) #ndata is the ts index of the last available time, reset the -1, then /NTIMES to get the day of the last available time.
    start = dt.datetime.strptime(start_dates.iloc[user][2]+" 00", '%Y-%m-%d %H')#.date()
    end = start + dt.timedelta(days=last_day)
    plotEnd = start + dt.timedelta(days=last_day+1)

    xs=np.array(range(len(result['varValues'])))
    availTimes = np.logical_and(~np.isnan(resultRun['reward']), resultRun['availability'] == 1)[:len(xs)]
    availTimes=np.where(resultRun['availability'] == 1)[0]
    availsAndNonNanReward=xs[availTimes]
    mapper=['red','blue']

    markersize=650
    opacity=.7
    colors=[mapper[int(i)] for i in result['varValues'][availTimes]]
    x = [ ]
    hourInc=[0,5,10,15,20]
    for i in range(NDAYS):
        for j in range(NTIMES):
            x.append((start + dt.timedelta(days=i)+dt.timedelta(hours=hourInc[j]))) 
    x=np.array(x)

    xs=x[availsAndNonNanReward]
    y=result['standardizedEffects'][availsAndNonNanReward]

    ax.set_ylim([-2, 2.75])
    vals=[-2,-1,0,1,2, 2.85]
    labs=['-2','-1','0','1','2','']
    ax.set_yticks(vals,labs)

    ax.axhline(y=0, color='k', linestyle='--', linewidth=6, alpha=.75)
    opacityB=.2
    yb=bs_results[len(bs_results)-1]['standardizedEffects'][availsAndNonNanReward]
    ax.scatter(xs[result['varValues'][availTimes] == 1], yb[result['varValues'][availTimes] == 1], marker='^', color='red',zorder=2, alpha=opacity,s=markersize)
    ax.scatter(xs[result['varValues'][availTimes] == 0], yb[result['varValues'][availTimes] == 0], marker='o', color='blue',zorder=2, alpha=opacity,s=markersize)
    
    colors=[mapper[int(i)] for i in result['varValues'][availTimes]]

    from matplotlib.lines import Line2D
    markersize=35
    lSize=40
    labelBlue=stateName+" = 0"
    labelRed=stateName+" = 1"
    legend_elements = [
                    Line2D([0], [0], marker='o', color='w', label=labelBlue,
                            markerfacecolor='b', markersize=markersize, alpha=opacity),
                    Line2D([0], [0], marker='^', color='w', label=labelBlue,
                            markerfacecolor='r', markersize=markersize, alpha=opacity)
                      ]
   
    ax.legend(handles=legend_elements, loc="upper left",fancybox = True, shadow = True, fontsize=lSize, handletextpad=-.2)

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)
    ax.grid(True, alpha=0.4)

    lSize=80
    ax.set_xlabel('Date (MM-DD)', fontsize=lSize)
    ax.set_ylabel("Std. Advantage", fontsize=lSize)

    lSize=75
    newLabels=[i.strftime('%m-%d') for i in xs]
    newXTicks=[]
    newLabels2=[]
    for i in range(len(x)):
        if i%(35*2)==0 and i!=0:
            newXTicks.append(x[i])
            newLabels2.append(x[i].strftime('%m-%d'))
    ax.set_xticks(newXTicks, newLabels2)
    ax.tick_params(axis='both', which='major', labelsize=lSize)
    ax.tick_params(axis='both', which='minor', labelsize=lSize)

    ax.set_xlim([start,plotEnd])
    myFmt = mdates.DateFormatter('%m-%d')
    ax.xaxis.set_major_formatter(myFmt)
    xticks = ax.xaxis.get_major_ticks()
    xticks[0].label1.set_visible(False)

    print(outputPath+'/blueRedOverAll_user-'+str(user)+'-state-'+stateName + "_resim-"+str(bs)+'.pdf')
    plt.tight_layout()
    plt.savefig(outputPath+'/blueRedOverAll_user-'+str(user)+'-state-'+stateName + "_resim-"+str(bs)+'.pdf')
    plt.clf()
    return 

def plotTrajectories(original_result, baseline, indexFS,gamma=.75,delta=.4):
    subset=2
    trajectories=random.sample(range(B), subset)
    bs_results=[]
    bs_results_sliding=[]
    for bootstrap in trajectories:
        print("processing "+str(bootstrap))
        output_dir = os.path.join("./output", "Baseline-"+ str(baseline)+"_UserSpecific-"+str(True)+"_User-"+str(i), "Bootstrap-" + str(bootstrap))
        onlyfiles = [f for f in listdir(output_dir) if isfile(join(output_dir, f))]
        if len(onlyfiles)>0:
            filepath=os.path.join(output_dir, onlyfiles[0])
            bs_res=pkl.load(open(filepath, "rb"))
            bs_results.append(bs_res)
            bs_results_sliding.append(computeMetricSlidingDay(bs_res, indexFS, delta1=gamma, delta2=delta))
            plotUserDayDateAndResims(resultSliding, original_result[i], i, "./plots_test", F_KEYS[indexFS], bs_results_sliding, bs_results, bootstrap)

B=500
for index in [2]:#[-1,2,3,4]: #for zero, engagement, location, and variation respectively
    r1s,r2s, rawR2s,baseline,interesting, r2_BS_Users, r2Raw_BS_Users, original_result=getResults(index)
    plot_C1_histogram(r1s, baseline)
    plot_C2_histogram(rawR2s, baseline)
    plot_heatmap_histogram(r1s, r2s, baseline, B=B)
    
    users=[4,77]
    plot_C2_User(r2_BS_Users, r2Raw_BS_Users, interesting, baseline, users)
    plot_Trajectories(original_result, baseline, index)